In [1]:
import requests
from yfiles_jupyter_graphs import GraphWidget
import webcolors
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color

# pip install requests webcolors colormath yfiles_jupyter_graphs jupyter_nbextensions_configurator

In [5]:
response = requests.post("http://0.0.0.0:8081/v1/filter/commits", json={
    "account": ???,
    "date_from": "2022-08-01",
    "date_to": "2022-09-01",
    "in": ["github.com/???"],
    "property": "everything",
}, headers={"Authorization": "Bearer ..."})

In [6]:
response.ok

True

In [9]:
commits = response.json()

In [13]:
commits, include = commits["data"], commits["include"]

In [156]:
w = GraphWidget()
w.nodes = [{
    "id": c["hash"],
    "properties": {
        "deployments": c.get("deployments", []),
        "timestamp": c["committer"]["timestamp"],
        "label": c["hash"],
    },
    "color": "blue",
} for c in commits]
w.edges = [{
    "start": c["hash"],
    "end": child,
} for c in commits for child in (c["children"] or [])]
w.set_sidebar(start_with="Search")
w.hierarchic_layout()
print("nodes:", len(w.nodes), "edges:", len(w.edges))

dag = {}
for c in commits:
    dag[c["hash"]] = c["children"] or []

nodes: 2765 edges: 3431


In [165]:
node_color_mappers = []


def normalize_to_rgb(c: str) -> webcolors.IntegerRGB:
    try:
        return webcolors.name_to_rgb(c)
    except ValueError:
        try:
            return webcolors.hex_to_rgb(c)
        except ValueError:
            assert c.startswith("rgb")
            return webcolors.IntegerRGB(*(int(p) for p in c.strip("rgba()").replace(" ", "").split(",")[:3]))


def average_color(*colors) -> str:
    if len(colors) == 1:
        return colors[0]
    colors = [convert_color(sRGBColor(*normalize_to_rgb(c), is_upscaled=True), LabColor) for c in colors]
    lab_l = sum(c.lab_l for c in colors) / len(colors)
    lab_a = sum(c.lab_a for c in colors) / len(colors)
    lab_b = sum(c.lab_b for c in colors) / len(colors)
    avg = convert_color(LabColor(lab_l, lab_a, lab_b), sRGBColor)
    return f"rgb({int(avg.rgb_r * 255)}, {int(avg.rgb_g * 255)}, {int(avg.rgb_b * 255)})"


def map_node_style(index: int, node: dict) -> str:
    colors = [c for mapper in node_color_mappers if (c := mapper(index, node)) is not None]
    if not colors:
        colors = [w.default_node_color_mapping(index, node)]
    return {
        "color": average_color(*colors),
        "shape": "round-rectangle",
    }

w.set_node_styles_mapping(map_node_style)

def highlight_deployment(name: str, color: str):
    def mapper(index, node):
        return color if name in node["properties"]["deployments"] else None
    node_color_mappers.append(mapper)


edge_color_mappers = []
edge_thickness_mappers = []


def map_edge_color(index: int, edge: dict) -> str:
    colors = [c for mapper in edge_color_mappers if (c := mapper(index, edge)) is not None]
    if not colors:
        colors = [w.default_edge_color_mapping(index, edge)]
    return average_color(*colors)


def map_edge_thickness(index: int, edge: dict) -> float:
    factors = [mapper(index, edge) for mapper in edge_thickness_mappers]
    if not factors:
        factors = [1.0]
    val = 1.0
    for f in factors:
        val *= f
    return val


w.set_edge_color_mapping(map_edge_color)
w.set_edge_thickness_factor_mapping(map_edge_thickness)


def highlight_path(node1: str, node2: str, color: str) -> tuple[str, str]:
    # we don't know the direction
    for node_from, node_to in ((node1, node2), (node2, node1)):
        visited = {}
        boilerplate = [(node_from, None)]
        while boilerplate:
            head, origin = boilerplate.pop(-1)
            if head in visited:
                continue
            visited[head] = origin
            if head == node_to:
                break
            boilerplate.extend((c, head) for c in reversed(dag.get(head, [])))
        if head == node_to:
            path = {head}
            while head := visited.get(head):
                path.add(head)

            def map_thickness(index, edge):
                if edge["start"] in path and edge["end"] in path:
                    return 4.0
                return 1.0

            def map_color(index, edge):
                if edge["start"] in path and edge["end"] in path:
                    return color
                return None

            edge_thickness_mappers.append(map_thickness)
            edge_color_mappers.append(map_color)
            print(node_from, f" -> ... {len(path) - 1} edges ... -> ", node_to)

In [166]:
node_color_mappers.clear()
edge_color_mappers.clear()
edge_thickness_mappers.clear()
highlight_deployment("???", "blue")
highlight_deployment("???", "red")
highlight_path("dc4e9cccc7cf4415be80505d33329b889349a96e", "821e1dcef8e0ce5241de419b66c0d768c33dea4d", "darkblue")

821e1dcef8e0ce5241de419b66c0d768c33dea4d  -> ... 173 edges ... ->  dc4e9cccc7cf4415be80505d33329b889349a96e


In [164]:
w

GraphWidget(layout=Layout(height='500px', width='100%'))